In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
from darts.models import ARIMA
import pandas as pd
import numpy as np

In [3]:
# Download source data from s3
from ml.common.folder_s3_sync import download_folder_from_s3

datapath = "../../../data/train_arima/"


download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1762686861', local_folder=datapath)
df = pd.read_csv(f'{datapath}Copper Ore.csv', delimiter=";")


Source: ost-s3/datasources/gw2/raw/1762686861
Target: ../../../data/train_arima/

Found 60 files to download

✓ +1 Agony Infusion.csv
✓ Auric Sliver.csv
✓ Bag of Coffee Beans.csv
✓ Bag of Sugar.csv
✓ Black Lion Chest.csv
✓ Bowl of Candy Corn Custard.csv
✓ Choya Spine.csv
✓ Congealed Putrescence.csv
✓ Copper Ore.csv
✓ Cup of Spiced Apple Cider.csv
✓ Elder Wood Log.csv
✓ Eye of Kormir.csv
✓ Flawless Snowflake.csv
✓ Fried Golden Dumpling.csv
✓ Glass of Buttered Spirits.csv
✓ Glob of Ectoplasm.csv
✓ Gold Ingot.csv
✓ Gold Ore.csv
✓ Gossamer Scrap.csv
✓ Green Wood Log.csv
✓ Handful of Red Lentils.csv
✓ Iron Ore.csv
✓ Jug of Water.csv
✓ Jute Scrap.csv
✓ Large Bone.csv
✓ Large Claw.csv
✓ Large Fang.csv
✓ Large Scale.csv
✓ Ley-Infused Sand.csv
✓ Lucent Mote.csv
✓ Lucky Draketail.csv
✓ Lucky Prismatic Rocket.csv
✓ Lump of Coal.csv
✓ Mithril Ore.csv
✓ Mystic Coin.csv
✓ Orichalcum Ore.csv
✓ Piece of Candy Corn.csv
✓ Piece of Common Unidentified Gear.csv
✓ Piece of Unidentified Gear.csv
✓ Piece of 

In [ ]:
from darts import TimeSeries
value_cols = ["buy_unit_price"]  #ARIMA only univariate !

tsdf = df[value_cols + ['fetched_at']].copy()
# series = TimeSeries.from_dataframe(df, time_col="fetched_at")

# Convert to datetime and set as index
tsdf['fetched_at'] = pd.to_datetime(tsdf['fetched_at'])
tsdf = tsdf.set_index('fetched_at')

# Resample to exact 5-minute intervals, forward-filling missing values
tsdf_resampled = tsdf[value_cols].resample('5min').mean().interpolate(method='linear')

# Create TimeSeries (no need for fill_missing_dates now)
series = TimeSeries.from_dataframe(tsdf_resampled, value_cols=value_cols)
series.plot();


In [ ]:
train, val = series.split_after(0.75)
train.plot(label="training")
val.plot(label="validation");


In [ ]:
from darts.models import NaiveDrift

drift_model = NaiveDrift()
drift_model.fit(train)
# Predict only the length of the validation set
drift_forecast = drift_model.predict(len(val))

series.plot()
drift_forecast.plot(label="drift");

In [ ]:
from darts.metrics import mape
from darts.models import ARIMA

best_mape = float('inf')
best_params = None

for p in range(0, 4):
    for d in range(0, 2):
        for q in range(0, 4):
            model = ARIMA(p=p, d=d, q=q)
            model.fit(train)
            forecast = model.predict(len(val))
            error = mape(val, forecast)

            if error < best_mape:
                best_mape = error
                best_params = (p, d, q)
                print(f"New best: ARIMA{best_params} with MAPE: {error:.2f}%")

print(f"Best model: ARIMA{best_params} with MAPE: {best_mape:.2f}%")

In [ ]:
from darts.models import Theta, NaiveDrift
from darts.metrics import mape


def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print(f"model {model} obtains MAPE: {mape(val, forecast):.2f}%")


eval_model(NaiveDrift())
eval_model(ARIMA())
eval_model(Theta())
eval_model(ARIMA(p=2, d=0, q=2))